<a href="https://colab.research.google.com/github/underwater/sam-witteveen-llm-tutorials/blob/main/colabs/YT_New_From_OpenAI_DevDay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install  ipykernel openai==1.1.1 tiktoken

In [ ]:
!pip show openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
import base64
import time
import openai
import os
import requests
from openai import OpenAI


## Basics GPT-4-turbo completion

In [ ]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are some of the key events that happened in Janurary 2023?",
        }
    ],
    model="gpt-4-1106-preview",
)

In [ ]:
print(completion.choices[0].message.content)

In [ ]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are some of the key events that happened Janurary 2023 for New Zealand Prime Minister Jacinda Ardern?",
        }
    ],
    model="gpt-3.5-turbo-1106",
)

In [ ]:
print(completion.choices[0].message.content)

## JSON Mode

In [ ]:

from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={"type": "json_object"},
  messages=[
    {"role": "system", "content": "You are a helpful programmer who always returns your answer in JSON."},
    {"role": "user", "content": "give me a list of 5 things for grocery shopping. call the list 'groceries'"}
  ]
)

print(completion.choices[0].message)

In [ ]:
groceries_list = json.loads(completion.choices[0].message.content)
print(groceries_list)
groceries_list['groceries'][4]


## New Function Calling

In [ ]:
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    print('get current weather called')
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": location, "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": location, "temperature": "72", "unit": "fahrenheit"})
    else:
        return json.dumps({"location": location, "temperature": "22", "unit": "celsius"})


def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in Tokyo?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    #get the message response
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls


    # Step 2: check if the model wanted to call a function
    print("step 2")
    if tool_calls:

        # Step 3: call the function
        print("step 3")
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        print("step 4")
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(f"Tool call {tool_call}")
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        print('step5')
        print(messages)
        second_response = openai.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response

print(run_conversation())

In [ ]:
get_current_weather("tokyo")

## DALL-E 3

In [ ]:
PROMPT = "a room full of cats all meditating in a circle"

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt= PROMPT,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
image_url

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display

def display_and_save_image_from_url(url, scale_percent=100, save_name='image.png'):
    # Send a GET request to the specified URL to retrieve the image
    response = requests.get(url)
    # Open the image
    img = Image.open(BytesIO(response.content))

    # Calculate the new size, as a percentage of the original size
    if scale_percent != 100:
        width, height = img.size
        new_width = int(width * scale_percent / 100)
        new_height = int(height * scale_percent / 100)
        img = img.resize((new_width, new_height), Image.ANTIALIAS)

    # Save the image locally with the given name
    img.save(save_name)

    # Display the image in the notebook
    display(img)

# Call the function with the URL, the scale percentage, and the save name you want
display_and_save_image_from_url(image_url, scale_percent=50, save_name='meditating_cats.png')

## GPT-V

In [ ]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "/content/meditating_cats.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Describe what’s in this image in detail as a story?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [ ]:
story = response.json()['choices'][0]['message']['content']

print(story)

## TTS

In [ ]:
from IPython.display import Audio
from pathlib import Path

response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=story
)

# Define the path where you want to save the file
speech_file_path = Path('/content/story.mp3')

# Save the response content (binary content of the mp3 file) to the path
with open(speech_file_path, 'wb') as file:
    file.write(response.content)

# Play the audio file
Audio(speech_file_path, autoplay=True)


## Deterministic outputs

In [ ]:
import asyncio
import openai
from openai import OpenAI
import pprint
import difflib
from IPython.display import display, HTML

GPT_MODEL = "gpt-3.5-turbo-1106"

In [ ]:
async def get_chat_response(system_message: str, user_request: str, seed: int = None):
    try:
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_request},
        ]


        client = OpenAI()

        completion = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            seed=seed,
            max_tokens=200,
            temperature=0.7,

        )

        # print(completion.choices[0].message)
        # print(completion.system_fingerprint)


        response_content = completion.choices[0].message.content #response["choices"][0]["message"]["content"]
        # print(f"response content: {response_content}")
        system_fingerprint = completion.system_fingerprint
        # print(f"system_fingerprint: {system_fingerprint}")
        prompt_tokens = completion.usage.prompt_tokens #response["usage"]["prompt_tokens"]
        completion_tokens = (
            completion.usage.total_tokens - prompt_tokens
        )

        table = f"""
        <table>
        <tr><th>Response</th><td>{response_content}</td></tr>
        <tr><th>System Fingerprint</th><td>{system_fingerprint}</td></tr>
        <tr><th>Number of prompt tokens</th><td>{prompt_tokens}</td></tr>
        <tr><th>Number of completion tokens</th><td>{completion_tokens}</td></tr>
        </table>
        """
        display(HTML(table))

        return response_content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# This function compares two responses and displays the differences in a table.
# Deletions are highlighted in red and additions are highlighted in green.
# If no differences are found, it prints "No differences found."


def compare_responses(previous_response: str, response: str):
    d = difflib.Differ()
    diff = d.compare(previous_response.splitlines(), response.splitlines())

    diff_table = "<table>"
    diff_exists = False

    for line in diff:
        if line.startswith("- "):
            diff_table += f"<tr style='color: red;'><td>{line}</td></tr>"
            diff_exists = True
        elif line.startswith("+ "):
            diff_table += f"<tr style='color: green;'><td>{line}</td></tr>"
            diff_exists = True
        else:
            diff_table += f"<tr><td>{line}</td></tr>"

    diff_table += "</table>"

    if diff_exists:
        display(HTML(diff_table))
    else:
        print("No differences found.")

In [ ]:
topic = "a journey to Mars"
system_message = "You are a helpful assistant that generates short stories."
user_request = f"Generate a short story about {topic}."

previous_response = await get_chat_response(
    system_message=system_message, user_request=user_request
)

response = await get_chat_response(
    system_message=system_message, user_request=user_request
)

# The function compare_responses is then called with the two responses as arguments.
# This function will compare the two responses and display the differences in a table.
# If no differences are found, it will print "No differences found."
compare_responses(previous_response, response)

In [ ]:
SEED = 123
response = await get_chat_response(
    system_message=system_message, seed=SEED, user_request=user_request
)
previous_response = response
response = await get_chat_response(
    system_message=system_message, seed=SEED, user_request=user_request
)

compare_responses(previous_response, response)

In [ ]:
responses = ''
for chunk in response:
    # print (chunk) # this will print all chunk objects
    # notice how we are converting it to string when concatenating
    if chunk. choices [0]. delta. content:
        text_chunk = chunk.choices [0].delta.content
        print (text_chunk, end-"", flush-"true")
        responses += str(text _chunk)
# print (responses)